## Neuzy HOC Model Tutorial ipynb

Unlike quick start, this "start" file contains explicit calling and argument fetching of the classes for Neuzy.
It can also serve as a handy introduction.

Importing the modules of Neuzy.
Import goes from PYTHONPATH which should be the root of the repository.

### Imports

In [ ]:
from neuzy.paropt.models.Parallizers import MPIpar          # Parallelization Class
from neuzy.paropt.models.HocModel import HocModel           # Model Class to read in HOC and have methods on the model
from neuzy.paropt.models.Stims import Firstspike_SortOutStim           # Stim protocol class
from neuzy.paropt.models.Calculations import FitnessCalcSD  # Calculations for Fitness
from neuzy.paropt.models.Optimizers import ScipyOpt         # Optimizer
from neuzy.paropt.models.Simulations import GenSim          # Run all objects in concatenation

### Creating the objects

#### Creating the HOC Model.
HocModel() inherits from GenModel().
For the full possible arguments that can be given, check the declaration/definition of HocModel.

In [ ]:
# Most arguments are optional
hoc_1 = HocModel(   
                        model_name = "Roe22_2.hoc",               # specify your model name or path, which is in /data/morphos folder.
                        modpath = None,                           # specify modpath, automatically set to data/mods/x86_64  - you have to compile them before usage with nrnivmodl.
                        target_feature_file = "somatic_features_hippounit.json", # specify your experimental or target feature data / json file 
                        template_name = "Roe22_reduced_CA1",      # optional, give template name of the HOC file
                        hippo_bAP = True,                         # updated soon TODO
                        channelblocknames = None,                 # sets the channels which are given in string or list of strings to 0
                        verbose = True,                           # prints a bit more
                        parameterkeywords = ["bar"]               # Parameter Keywords from psection() density mechs, which are to be used.  
                        )

#### Parallizer MPI object to set the parallelization method to MPI

In [ ]:
mpi_1 = MPIpar( populationsize = 100 )    # Generates up to 100 optimized models that are within the cost thresholds.

#### Stim Object, where stimulation parameters are defined for NEURON

Using Firstspike_SortOutStim() which inherits from GenStim() to also include a faster way to check if a model spikes sufficiently before going into the optimizer. See documentation.

In [ ]:
## Stimulation object with stimulation delay, duration, timestop.

# Stepamps and Clamp which should be used. 

# Have to be in accordance to your target feature data, otherwise you cannot compare features for the same step amplitude.
stepamps = {'Step-04': -0.4, 'Step08' : 0.8} 
clamp = 'IClamp'        # name should equal a HocObject method to define clamp. E.g. h.IClamp or 'VClamp' for h.VClamp

stim_1 = Firstspike_SortOutStim(    # inherited from GenStim()
                                    delay = 150,            
                                    duration = 400,         
                                    tstop = 750,            
                                    cvode_active = None,    
                                    stepamps = stepamps,    
                                    clamp = clamp,          

                                    section_locations_dict = None,    # TODO, automatically create a dictionary with recording vectors for all sections of the morphology

                                    # FirstSpike_SortOutStim parameters for very fast stimulation, that only checks for sufficient first AP of a trace.
                                    # i.e. models have to spike to be seen as good enough for full stimulations.
                                    delay_firstspike = 50, 
                                    duration_firstspike = 65, 
                                    tstop_firstspike = 150    
                                    )    

#### Calculation Object, to determine which Calculation should be chosen.
FitnessCalcSD() inherits from GenCalc() and calculates the Cost/Fitness values depending on SD.

In [ ]:
calc_1 = FitnessCalcSD()

#### Optimizer Object, to determine which Optimizer should be chosen.
ScipyOpt() inherits from GenOpt().

In [ ]:
## Optimizer used from Scipy - hence the name. 
# TODO add more options, e.g. DEAP

opt_1 = ScipyOpt( 
                    method = "Nelder-Mead",             # Optimization with Nelder-Mead algorithm, also 'L-BFGS-B' or Conjugate Gradient 'CG'
                    init_cost_threshold = 2,            # Cost threshold, which immediately sees a parameter combination of the model as sufficient  
                    init_cost_optimizer_threshold = 5   # Cost threshold, which is low enough to be further optimized. Models above are disregarded and recreated.
                    )         

#### Putting them all together
GenSim() is the general Simulation class, which runs all defined objects combined.

In [ ]:
## Running default config
run_instance_1 = GenSim(par = mpi_1, model = hoc_1, stim = stim_1, opt = opt_1, calc = calc_1)

## Call the objects together
run_instance_1.run()

#### Outputs

Check log files or terminal output to see the status of the run.
Enjoy.